In [73]:
import numpy as np
import numpy.linalg
import pandas as pd 

In [74]:
def price (n, sigma, r, y, dt, ds):
  matrix = np.zeros((n, n))
  matrix[0, 0] = 1 + r * dt + pow(sigma, 2) * pow(0, 2) * dt
  matrix[0, 1] = - pow(sigma, 2) * pow(0, 2) * dt / 2 + (r - y) * 0 * dt / 2
  matrix[n-1, n-1] = 1 + r * dt + pow(sigma, 2) * pow(n - 1, 2) * dt
  matrix[n-1, n-2] = - pow(sigma, 2) * pow(n - 1, 2) * dt / 2 - (r - y) * (n - 1) * dt / 2

  for i in range(1, n-1):
    matrix[i, i - 1] = - pow(sigma, 2) * pow(i, 2) * dt / 2 - (r - y) * i * dt / 2
    matrix[i, i] = 1 + r * dt + pow(sigma, 2) * pow(i, 2) * dt
    matrix[i, i + 1] = - pow(sigma, 2) * pow(i, 2) * dt / 2 + (r - y) * i * dt / 2
  
  grid = []
  T = int(5 / dt)
  D = int(1 / dt)
  
  K_T = numpy.linalg.inv(matrix)

  # First boundary
  W = np.transpose(np.mat(np.maximum(1000, np.arange(n) * ds * 1000 / 12))) + 60 * (T - 1 / dt) * dt
  F = np.dot(K_T, W)
  grid.append(F)
  
  # Second and third boundary
  for i in range(T - 1, D - 1, -1):
    F = np.dot(K_T, F)
    S_t = np.transpose((np.mat(np.arange(n) * ds * 1000 / 12)))
    S_t[:1600] += 60 * (i - 1 / dt + 1) * dt
    S_t[1600:] += 60 * 4
    F[0] = 0
    F = np.maximum(F, S_t)
    grid.append(F)

  for i in range(D - 1, 0, -1):
    F=np.dot(K_T,F)
    S_t = np.transpose((np.mat(np.arange(n) * ds * 1000 / 12)))
    F[0] = 0
    F = np.maximum(F, S_t)
    grid.append(F)

  return grid[::-1]

In [75]:
# Q1

sigma = 2
r = 0.01
y = 0.05
dt = 1 / 52
S = 60
ds = 0.04
n = int(S / ds)
F = price(n, sigma, r, y, dt, ds)
p = F[0][int(10 / ds)]
print(F[0][int(10 / ds)])

[[970.13916081]]


In [76]:
# Sensitivity

ds = 0.04
# 1
S1 = 55  
n = int(S1 / ds)
F = price(n, sigma, r, y, dt, ds)
print(F[0][int(10 / ds)])

[[959.57546438]]


In [77]:
# 2
S2 = 58
n = int(S2 / ds)
F = price(n, sigma, r, y, dt, ds)
print(F[0][int(10 / ds)])

[[966.06594643]]


In [78]:
# 3
S3 = 63
n = int(S3 / ds)
F = price(n, sigma, r, y, dt, ds)
print(F[0][int(10 / ds)])

[[975.9062265]]


In [79]:
# 4
S4 = 65
n = int(S4 / ds)
F = price(n, sigma, r, y, dt, ds)
print(F[0][int(10 / ds)])

[[979.54232432]]


In [80]:
S = 60
# 1 
ds1 = 0.03  
n = int(S / ds1)
F = price(n, sigma, r, y, dt, ds1)
print(F[0][int(10 / ds1)])

[[984.6949495]]


In [82]:
# 2
ds2 = 0.035 
n = int(S / ds2)
F = price(n, sigma, r, y, dt, ds2)
print(F[0][int(10 / ds2)])

[[975.17787794]]


In [83]:
# 3
ds3 = 0.045 
n = int(S / ds3)
F = price(n, sigma, r, y, dt, ds3)
print(F[0][int(10 / ds3)])

[[963.41074475]]


In [85]:
# 4
ds4 = 0.05
n = int(S / ds4)
F = price(n, sigma, r, y, dt, ds4)
print(F[0][int(10 / ds4)])

[[958.99427076]]


In [86]:
# Question 2

y2 = 0.2
S = 60
ds = 0.04
n = int(S / ds)
F = price(n, sigma, r, y2, dt, ds)
p2 = F[0][int(10 / ds)]
print(F[0][int(10 / ds)])
print(p2 - p)

[[1057.10017344]]
[[86.96101262]]


In [95]:
# Question 3

sigma = 2
r = 0.01
y = 0.05
dt = 1 / 260
S = 55
ds = 0.05
n = int(S / ds)
F = price(n, sigma, r, y, dt, ds)
print(F[0][int(10 / ds)])
shares = 10000000 / F[0][int(10 / ds)]
print(shares / 1000)

[[993.59669766]]
[[10.06444569]]


In [52]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving MCP_stock_price.xlsx to MCP_stock_price (2).xlsx
Saving Molycorp_bond_price.xlsx to Molycorp_bond_price (2).xlsx
User uploaded file "MCP_stock_price.xlsx" with length 43619 bytes
User uploaded file "Molycorp_bond_price.xlsx" with length 26116 bytes


In [57]:
bond_price = pd.read_excel('Molycorp_bond_price.xlsx')[['Date', 'Last']]
stock_price = pd.read_excel('MCP_stock_price.xlsx')[['Date', 'Adj Close']]
bond_price['Date'] = pd.to_datetime(bond_price['Date'])
stock_price['Date'] = pd.to_datetime(stock_price['Date'])
all_data = (pd.merge(bond_price, stock_price, left_on='Date', right_on='Date', how='right', sort=True)).interpolate(method='linear')
print(all_data)
bond = all_data['Last']
stock = all_data['Adj Close']
delta = np.zeros(len(all_data) - 1)
hedge = np.zeros(len(all_data) - 1)
unhedge = np.zeros(len(all_data) - 1)

          Date        Last  Adj Close
0   2012-08-17   99.304001       9.84
1   2012-08-20  101.000000      10.02
2   2012-08-21   98.875198       9.84
3   2012-08-22   97.750000       9.64
4   2012-08-23   99.241997       9.68
..         ...         ...        ...
776 2015-09-21    0.266767       0.10
777 2015-09-22    0.200150       0.09
778 2015-09-23    0.133533       0.10
779 2015-09-24    0.066917       0.10
780 2015-09-25    0.000300       0.10

[781 rows x 3 columns]


In [98]:
for i in range(len(all_data) - 1):
  delta[i] = (F[i][int(stock[i] / ds) + 1] - F[i][int(stock[i] / ds) - 1]) / (2 * ds)

for i in range(1, len(all_data) - 1):
  hedge[i] = -delta[i - 1] * (stock[i] - stock[i - 1]) + (bond[i] - bond[i-1]) * shares / 1000
  unhedge[i] = (bond[i] - bond[i - 1]) * shares / 1000

print(np.mean(hedge), np.std(hedge))
print(np.mean(unhedge), np.std(unhedge))


-0.1731130602237301 21.23464616393026
-1.280469543709268 17.44138066908753
